In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import math
import random
from tqdm import tqdm_notebook
import matplotlib
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches

from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 30)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from datetime import datetime
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
path='../input/ashrae-energy-prediction/'
path_new = '../input/transformer/'
from sklearn.metrics import roc_auc_score,mean_squared_error
from sklearn.model_selection import StratifiedKFold,TimeSeriesSplit, train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from math import sin,log,pow
import lightgbm as lgb
import datetime
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder,StandardScaler, OneHotEncoder, MinMaxScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import multiprocessing as mp
from multiprocessing import Pool
from functools import partial
pd.options.mode.chained_assignment = None
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
import holidays
# from meteocalc import feels_like, Temp
from itertools import groupby
from operator import itemgetter

import os
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))
def ls():
     print([v for v in globals().keys() if not v.startswith('_')])


../input/ashrae-energy-prediction/train.csv
../input/ashrae-energy-prediction/building_metadata.csv
../input/ashrae-energy-prediction/sample_submission.csv
../input/ashrae-energy-prediction/weather_test.csv
../input/ashrae-energy-prediction/weather_train.csv
../input/ashrae-energy-prediction/test.csv


In [2]:
####### useful function for cleaning part###

def input_file(file):
    path = f"../input/ashrae-energy-prediction/{file}"
    if not os.path.exists(path): return path + ".gz"
    return path

def compress_dataframe(df):
    result = df.copy()
    for col in result.columns:
        col_data = result[col]
        dn = col_data.dtype.name
        if dn == "object":
            result[col] = pd.to_numeric(col_data.astype("category").cat.codes)
        elif dn == "bool":
            result[col] = col_data.astype("int8")
        elif dn.startswith("int") or (col_data.round() == col_data).all():
            result[col] = pd.to_numeric(col_data)
        else:
            result[col] = pd.to_numeric(col_data)
    return result

def read_train():
    df = pd.read_csv(input_file("train.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    return compress_dataframe(df)

def read_building_metadata():
    return compress_dataframe(pd.read_csv(
        input_file("building_metadata.csv")).fillna(-1)).set_index("building_id")

site_GMT_offsets = [-5, 0, -7, -5, -8, 0, -5, -5, -5, -6, -7, -5, 0, -6, -5, -5]

def read_weather_train(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    df = pd.read_csv(input_file("weather_train.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    if fix_timestamps:
        GMT_offset_map = {site: offset for site, offset in enumerate(site_GMT_offsets)}
        df.timestamp = df.timestamp + df.site_id.map(GMT_offset_map)
    if interpolate_na:
        site_dfs = []
        for site_id in df.site_id.unique():
            # Make sure that we include all possible hours so that we can interpolate evenly
            site_df = df[df.site_id == site_id].set_index("timestamp").reindex(range(8784))
            site_df.site_id = site_id
            for col in [c for c in site_df.columns if c != "site_id"]:
                if add_na_indicators: site_df[f"had_{col}"] = ~site_df[col].isna()
                site_df[col] = site_df[col].interpolate(limit_direction='both', method='linear')
                # Some sites are completely missing some columns, so use this fallback
                site_df[col] = site_df[col].fillna(df[col].median())
            site_dfs.append(site_df)
        df = pd.concat(site_dfs).reset_index()  # make timestamp back into a regular column
    elif add_na_indicators:
        for col in df.columns:
            if df[col].isna().any(): df[f"had_{col}"] = ~df[col].isna()
    return compress_dataframe(df).set_index(["site_id", "timestamp"])

def combined_train_data(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    Xy = compress_dataframe(read_train().join(read_building_metadata(), on="building_id").join(
        read_weather_train(fix_timestamps, interpolate_na, add_na_indicators),
        on=["site_id", "timestamp"]).fillna(-1))
    return Xy.drop(columns=["meter_reading"]), Xy.meter_reading


In [3]:
###### alghorithm to clean data ####
## it find outlier for 1999 build, and strange pattern, depending on the meter and month considered.

def make_is_bad_zero(Xy_subset, min_interval=48, summer_start=3000, summer_end=7500):
    """Helper routine for 'find_bad_zeros'.
    
    This operates upon a single dataframe produced by 'groupby'. We expect an 
    additional column 'meter_id' which is a duplicate of 'meter' because groupby 
    eliminates the original one."""
    meter = Xy_subset.meter_id.iloc[0]
    is_zero = Xy_subset.meter_reading == 0
    if meter == 0:
        # Electrical meters should never be zero. Keep all zero-readings in this table so that
        # they will all be dropped in the train set.
        return is_zero

    transitions = (is_zero != is_zero.shift(1))
    all_sequence_ids = transitions.cumsum()
    ids = all_sequence_ids[is_zero].rename("ids")
    if meter in [2, 3]:
        # It's normal for steam and hotwater to be turned off during the summer
        keep = set(ids[(Xy_subset.timestamp < summer_start) |
                       (Xy_subset.timestamp > summer_end)].unique())
        is_bad = ids.isin(keep) & (ids.map(ids.value_counts()) >= min_interval)
    elif meter == 1:
        time_ids = ids.to_frame().join(Xy_subset.timestamp).set_index("timestamp").ids
        is_bad = ids.map(ids.value_counts()) >= min_interval

        # Cold water may be turned off during the winter
        jan_id = time_ids.get(0, False)
        dec_id = time_ids.get(8283, False)
        if (jan_id and dec_id and jan_id == time_ids.get(500, False) and
                dec_id == time_ids.get(8783, False)):
            is_bad = is_bad & (~(ids.isin(set([jan_id, dec_id]))))
    else:
        raise Exception(f"Unexpected meter type: {meter}")

    result = is_zero.copy()
    result.update(is_bad)
    return result

def find_bad_zeros(X, y):
    """Returns an Index object containing only the rows which should be deleted."""
    Xy = X.assign(meter_reading=y, meter_id=X.meter)
    is_bad_zero = Xy.groupby(["building_id", "meter"]).apply(make_is_bad_zero)
    return is_bad_zero[is_bad_zero].index.droplevel([0, 1])

def find_bad_sitezero(X):
    """Returns indices of bad rows from the early days of Site 0 (UCF)."""
    return X[(X.timestamp < 3378) & (X.site_id == 0) & (X.meter == 0)].index

def find_bad_building1099(X, y):
    """Returns indices of bad rows (with absurdly high readings) from building 1099."""
    return X[(X.building_id == 1099) & (X.meter == 2) & (y > 3e4)].index

def find_bad_rows(X, y):
    bad_zeros = find_bad_zeros(X, y)
    gc.collect()
    bad_site = find_bad_sitezero(X)
    gc.collect()
    bad_build = find_bad_building1099(X, y)
    gc.collect()
    return bad_zeros.union(bad_site).union(bad_build)

def bad_rows_dropper(df):
    print('Load data...\n')
    X, y = combined_train_data()
    if X.shape[0]!=df.shape[0]:
        raise ValueError('Number of rows for cleaning are different\n')
    if (np.abs(y-df.meter_reading)).max()>1e-5:
        raise ValueError('Target is different\n')
    print('Find rows...\n')
    bad_rows = find_bad_rows(X, y)
    print(f'Number of rows dropped: {len(bad_rows)}')
    return(df.drop(bad_rows,axis=0).reset_index(drop=True))

In [4]:
def build_sur_area(x,h):
    side = x**(0.5)
    return 2*(side*side + side*h + side*h)

def evaporated_water(x0, x1, x2, x3, x4, x5, time):
    xs = 0.622*(((x0*x1*0.026325) / x3)-1)  # maxm humidity ratio of saturated air
    x = 0.622 * (((x0 * x2 * 0.026325) / x3) - 1)  # humidity ratio of dry air
    if time=='hour': return x4 * x5 *(xs-x)
    if time=='minute': return (x4 * x5 *(xs-x))/60
    if time=='sec': return (x4 * x5 *(xs-x))/3600

# heat required by building
def required_by_build(x0, x1, x2):
    return((2256+((x0**(1/3)+1)**3 - x0)*x1)*x2)

def heat_require(sur,delta):
    return(sur*delta)

def strater_feat(x):
    strat =x['building_id'].astype(str) + x['meter'].astype(str)
    le = LabelEncoder()
    strat = le.fit_transform(strat).astype(np.int16)
    del le
    return(strat)

def diff_meter(df):
    df['diff_meter'] = df['building_id'].map(df.groupby(['building_id'])['meter'].unique()).apply(lambda x : np.array_str(np.sort(x)))

def fe_df(df, char = ['precip_depth_1_hr','wind_load','build_sur_area',
                      'air_density','wind_direction','wind_speed','temp_diff','relative_humidity','wind_load']):
    df['air_density'] = (((df['sea_level_pressure'].fillna(980))/69.0695296524) / ((df['air_temperature']+273.15) * 0.0821)).astype(np.float16)

    # Building surface area
    df['building_height'] = (df['floor_count'].replace(np.nan,1) * 3).astype(np.int16)  # each floor 3m high
    df['building_vol'] = (df['building_height'] * df['square_feet']).astype(np.float32)

    df['build_sur_area'] = (build_sur_area(df['square_feet'], df['building_height'])).astype(np.float32)
    
    df.drop(['building_height'],axis=1,inplace=True)
    gc.collect()

    df['heating_required'] = (heat_require(df['build_sur_area'],(df['air_temperature']-20))).astype(np.float32)
    return(df.drop(char,axis=1))
    

def sd(col, max_loss_limit=0.001, avg_loss_limit=0.001, na_loss_limit=0, n_uniq_loss_limit=0, fillna=0):
    """
    max_loss_limit - don't allow any float to lose precision more than this value. Any values are ok for GBT algorithms as long as you don't unique values.
                     See https://en.wikipedia.org/wiki/Half-precision_floating-point_format#Precision_limitations_on_decimal_values_in_[0,_1]
    avg_loss_limit - same but calculates avg throughout the series.
    na_loss_limit - not really useful.
    n_uniq_loss_limit - very important parameter. If you have a float field with very high cardinality you can set this value to something like n_records * 0.01 in order to allow some field relaxing.
    """
    is_float = str(col.dtypes)[:5] == 'float'
    na_count = col.isna().sum()
    n_uniq = col.nunique(dropna=False)
    try_types = ['float16', 'float32']

    if na_count <= na_loss_limit:
        try_types = ['int8', 'int16', 'float16', 'int32', 'float32']

    for type in try_types:
        col_tmp = col

        # float to int conversion => try to round to minimize casting error
        if is_float and (str(type)[:3] == 'int'):
            col_tmp = col_tmp.copy().fillna(fillna).round()

        col_tmp = col_tmp.astype(type)
        max_loss = (col_tmp - col).abs().max()
        avg_loss = (col_tmp - col).abs().mean()
        na_loss = np.abs(na_count - col_tmp.isna().sum())
        n_uniq_loss = np.abs(n_uniq - col_tmp.nunique(dropna=False))

        if max_loss <= max_loss_limit and avg_loss <= avg_loss_limit and na_loss <= na_loss_limit and n_uniq_loss <= n_uniq_loss_limit:
            return col_tmp

    # field can't be converted
    return col


def reduce_mem_usage_sd(df, deep=True, verbose=False, obj_to_cat=False):
    numerics = ['int16', 'uint16', 'int32', 'uint32', 'int64', 'uint64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=deep).sum() / 1024 ** 2
    for col in tqdm_notebook(df.columns):
        col_type = df[col].dtypes

        # collect stats
        na_count = df[col].isna().sum()
        n_uniq = df[col].nunique(dropna=False)
        
        # numerics
        if col_type in numerics:
            df[col] = sd(df[col])

        # strings
        if (col_type == 'object') and obj_to_cat:
            df[col] = df[col].astype('category')
        
        if verbose:
            print(f'Column {col}: {col_type} -> {df[col].dtypes}, na_count={na_count}, n_uniq={n_uniq}')
        new_na_count = df[col].isna().sum()
        if (na_count != new_na_count):
            print(f'Warning: column {col}, {col_type} -> {df[col].dtypes} lost na values. Before: {na_count}, after: {new_na_count}')
        new_n_uniq = df[col].nunique(dropna=False)
        if (n_uniq != new_n_uniq):
            print(f'Warning: column {col}, {col_type} -> {df[col].dtypes} lost unique values. Before: {n_uniq}, after: {new_n_uniq}')

    end_mem = df.memory_usage(deep=deep).sum() / 1024 ** 2
    percent = 100 * (start_mem - end_mem) / start_mem
    print('Mem. usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)'.format(start_mem, end_mem, percent))
    return df


def chunk_predictor(X,model,n=100,verbose=True):
    predict_fold = KFold(n)
    result = np.zeros(X.shape[0])
    if verbose:
        for trn_idx, val_idx in tqdm_notebook(predict_fold.split(X)):
               result[val_idx] = model.predict(X.iloc[val_idx,:])
    else:
        for trn_idx, val_idx in predict_fold.split(X):
            result[val_idx] = model.predict(X.iloc[val_idx,:])
    return(result)

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

def debugger_train(train,target):
    weather_col = int(np.where(train.columns == 'air_temperature')[0])
    mask = (train.iloc[:,weather_col:train.shape[1]].isnull().mean(axis=1)==1)

    if train.shape[0]!=len(target):
        raise ValueError('Shape from train and target are different')
    if any(mask):
        raise ValueError('Missing row in weather')
    return(train.columns)

def debugger_test(test,col):
    if len(col)!=test.shape[1]:
        raise ValueError('Shape from train and target are different')
    if np.any(test.columns!=col):
        raise ValueError('Test column are different with train')
        
def aligner_holiday(weather):
    timediff = {0:5,1:0,2:7,3:5,4:8,5:0,6:5,7:5,8:5,9:6,10:7,11:5,12:0,13:6,14:5,15:5}#{0:4,1:0,2:7,3:4,4:7,5:0,6:4,7:4,8:4,9:5,10:7,11:4,12:0,13:5,14:4,15:4}
    weather['time_diff'] = weather['site_id'].map(timediff)
    weather['time_diff'] = pd.to_timedelta(weather['time_diff'],'h')
    weather['timestamp'] = weather['timestamp'] - weather['time_diff']
    weather = city_mapper(weather)
    weather = holiday_mapper(weather)
    
    country_dic = {'UnitedStates':0,'England':1,'Canada':2,'Ireland':3} 
    holiday_dic = {False:0,True:1}
    
    weather['country'] = weather['country'].map(country_dic)
    weather['Holiday'] = weather['Holiday'].map(holiday_dic)
    del timediff,country_dic,holiday_dic
    return(weather.drop(['city','time_diff'],axis=1))


def city_mapper(weather):    
    country = {0:'UnitedStates',1:'England',2:'UnitedStates',3:'UnitedStates',4:'UnitedStates',
           5:'England',6:'UnitedStates',7:'Canada',8:'UnitedStates',9:'UnitedStates',10:'UnitedStates',
           11:'Canada',12:'Ireland',13:'UnitedStates',14:'UnitedStates',15:'UnitedStates'}
    city = {0:'Jacksonville',1:'London',2:'Phoenix',3:'Philadelphia',4:'San Francisco',
           5:'Loughborough',6:'Philadelphia',7:'Montreal',8:'Jacksonville',9:'San Antonio',10:'Las Vegas',
           11:'Montreal',12:'Dublin',13:'Minneapolis',14:'Philadelphia',15:'Pittsburgh'} 
    weather['country'] = weather['site_id'].map(country)
    weather['city'] = weather['site_id'].map(city)
    del country,city
    return(weather)

def holiday_mapper(weather):
    state = {'Jacksonville':'FL','London':None,'Phoenix':'AZ','Philadelphia':'PA','San Francisco':'CA',
     'Loughborough':None,'Philadelphia':'PA','Montreal':'QC','Jacksonville':'FL',
     'San Antonio':'TX','Las Vegas':'NV','Montreal':'QC','Dublin':None,'Minneapolis':'MN','Philadelphia':'PA',
     'Pittsburgh':'PA'}

    city_country_dic = {'Jacksonville':'UnitedStates','London':'England','Phoenix':'UnitedStates','Philadelphia':'UnitedStates',
                        'San Francisco':'UnitedStates','Loughborough':'England','Philadelphia':'UnitedStates',
                        'Montreal':'Canada','Jacksonville':'UnitedStates','San Antonio':'UnitedStates',
                        'Las Vegas':'UnitedStates','Montreal':'Canada',
                        'Dublin':'Ireland','Minneapolis':'UnitedStates','Philadelphia':'UnitedStates','Pittsburgh':'UnitedStates'}

    weather['Holiday'] = True
    for (city,abbr) in state.items():

        if city_country_dic[city] == 'UnitedStates':
            holiday = holidays.CountryHoliday(city_country_dic[city],state=abbr)
        else:
            holiday = holidays.CountryHoliday(city_country_dic[city],prov=abbr)

        weather.loc[weather.city==city,'Holiday'] = [x in holiday for x in weather.loc[weather.city==city,'timestamp']]
    del state,city_country_dic,holiday
    return(weather)

def clean_timestamps(df):
    """
    :param df: Dataframe containing a "timestamp" field which will be broken down in hour, year, day,...
    """
    df['timestamp'] = pd.to_datetime(df['timestamp'],format='%Y-%m-%d %H:%M:%S')
    df["day"] = df["timestamp"].dt.day.astype(np.uint8)
    df["hour"] = df["timestamp"].dt.hour.astype(np.uint8)
    df["weekend"] = df["timestamp"].dt.weekday.astype(np.uint8)
    
def free_space(del_list):
    for name in del_list:
        if not name.startswith('_'):
            del globals()[name]
    gc.collect()

def vector_apply(vec,fun):
    result = vec.copy()
    for i in tqdm_notebook(range(len(vec))):
        result[i] = fun(vec[i])
    return(result)

def res_fun(x):
    return(truncate(math.exp(x)-1,4))


def leak_to_train():
    ucf_root = '../input/ashrae-ucf-spider-and-eda-full-test-labels/'
    ucl_root ='../input/ucl-data-leakage-episode-2/'
    asu_root ='../input/asu-kernel-leach/'
    
    leak_df_ucf = pd.read_hdf(f'{ucf_root}site0.h5',key='site0')
#     leak_df_ucf = pd.read_pickle(f'{ucf_root}site0.pkl') 
    leak_df_ucf['meter_reading'] = leak_df_ucf.meter_reading_scraped
    leak_df_ucf.drop(['meter_reading_original','meter_reading_scraped'], axis=1, inplace=True)
    leak_df_ucf = leak_df_ucf[~(leak_df_ucf['meter_reading'].isnull())]
    leak_df_ucf.loc[leak_df_ucf.meter_reading < 0, 'meter_reading'] = 0
    leak_df_ucf = leak_df_ucf[leak_df_ucf.timestamp.dt.year > 2016]
    leak_df_ucf = leak_df_ucf[['timestamp','building_id','meter','meter_reading']]
    leak_df_ucf.reset_index(drop=True,inplace=True)
    
    leak_df_ucl = pd.read_hdf(f'{ucl_root}site1.h5',key='site1')
#     leak_df_ucl = pd.read_pickle(f'{ucl_root}site1.pkl') 
    leak_df_ucl['meter_reading'] = leak_df_ucl.meter_reading_scraped
    leak_df_ucl.drop(['meter_reading_scraped'], axis=1, inplace=True)
    leak_df_ucl = leak_df_ucl[~(leak_df_ucl['meter_reading'].isnull())]
    leak_df_ucl.loc[leak_df_ucl.meter_reading < 0, 'meter_reading'] = 0
    leak_df_ucl = leak_df_ucl[leak_df_ucl.timestamp.dt.year > 2016]
    leak_df_ucl = leak_df_ucl[['timestamp','building_id','meter','meter_reading']]
    leak_df_ucl.reset_index(drop=True,inplace=True)
    
    
    leak_df_asu = pd.read_hdf(f'{asu_root}site2_scraped_pd.h5',key='site2')
#     leak_df_asu = pd.read_pickle(f'{asu_root}site2_scraped_pd.pkl')
    leak_df_asu['timestamp'] = pd.to_datetime(leak_df_asu['timestamp'],format='%Y-%m-%d %H:%M:%S')
    leak_df_asu = leak_df_asu[~(leak_df_asu['meter_reading'].isnull())]
    leak_df_asu.loc[leak_df_asu.meter_reading < 0, 'meter_reading'] = 0
    leak_df_asu = leak_df_asu[leak_df_asu.timestamp.dt.year > 2016]
    leak_df_asu = leak_df_asu[['timestamp','building_id','meter','meter_reading']]
    leak_df_asu.reset_index(drop=True,inplace=True)

    leak_df = pd.concat((leak_df_ucf,leak_df_ucl,leak_df_asu),axis=0)
    leak_df.drop_duplicates(inplace=True)
    leak_df.sort_values(['timestamp','building_id','meter'],inplace=True)
    leak_df.reset_index(drop=True,inplace=True)
    return(leak_df)

def leaked_concat(train):
    leak_df = leak_to_train()
    train = pd.concat((train,leak_df),axis=0)
    train.drop_duplicates(inplace=True)
    train.sort_values(['timestamp','building_id','meter'],inplace=True)
    train.reset_index(drop=True,inplace=True)
    del leak_df
    return(train)

def temp_square_feet(x):
    x['m2k'] = x['square_feet']*x['air_temperature']
    x['pm2k'] = (1/x['m2k']).replace([-np.inf,np.inf],np.nan)
    return(x)

def to_del(x,char=['country','cloud_coverage','beaufort_scale','wind_compass','Holiday','sea_level_pressure',
                  'precip_depth_1_hr','wind_load','wind_chill',
                         'wind_direction','wind_speed','temp_diff','relative_humidity','wind_load']):
    char = [y for y in char if y in x.columns]
    return(x.drop(char,axis=1))

def reduce_num(x,char = ['precip_depth_1_hr','wind_load',
                         'wind_direction','wind_speed','temp_diff','relative_humidity','wind_load'],component=1):
    scaler = StandardScaler()
    temp = x[char]
    temp = temp.apply(lambda x: x.fillna(x.mean())) 
    scaler = StandardScaler()
    temp = scaler.fit_transform(temp)
    pca = PCA(n_components=component)
    pca.fit(temp)
    print('PCA explained-variance: {}'.format(sum(pca.explained_variance_ratio_)))
    fitted = pca.transform(temp).astype(np.float32)
    for col in range(component):
        x['pca_num'+str(col)] = fitted[:,col]
    return((x.drop(char,axis=1)))
    
def aggr_build(x):
    train = pd.read_hdf(f'{path_new}data.h5',key='train')
    train['timestamp'] = pd.to_datetime(train['timestamp'],format='%Y-%m-%d %H:%M:%S')
    train['meter_reading'] = np.log1p(train['meter_reading'])
    
    df_group = train.groupby(['building_id','meter'])['meter_reading']
    building_mean = df_group.mean().astype(np.float16).rename('meter_mean').reset_index()
    building_median = df_group.median().astype(np.float16).rename('meter_median').reset_index()
    building_std = df_group.std().astype(np.float16).rename('meter_std').reset_index()

    x = x.merge(building_mean, left_on = ['building_id','meter'], right_on = ['building_id','meter'], how = "left")
    x = x.merge(building_median, left_on = ['building_id','meter'], right_on = ['building_id','meter'], how = "left")
    x = x.merge(building_std, left_on = ['building_id','meter'], right_on = ['building_id','meter'], how = "left")
    del train, df_group, building_median, building_mean, building_std
    return(x)


def add_lag_feature(weather_df,cols =['air_temperature','feel'], window=24): 
    group_df = weather_df.groupby('site_id')
    rolled = group_df[cols].rolling(window=window, min_periods=0)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    for col in cols:
        weather_df[f'{col}_mean_lag{window}'] = lag_mean[col]
    del group_df,rolled,lag_mean
    return(weather_df)

# def counter_pre_work():
#     building_metadata = pd.read_hdf(f'{path_new}data.h5',key='building_metadata')[['building_id','primary_use']]
#     temp = pd.read_hdf(f'{path_new}data.h5',key='train',columns = [['building_id','meter']])
#     temp2 = pd.read_hdf(f'{path_new}data.h5',key='test',columns = [['building_id','meter']])
#     supp = reduce_mem_usage_sd(temp2.append(temp,ignore_index =True))
#     del temp, temp2
#     gc.collect()
#     supp = strater_feat(supp)
#     supp = reduce_mem_usage_sd(supp.merge(building_metadata, left_on = "building_id", right_on = "building_id", how = "left"))
#     supp['diff_meter'] = supp['building_id'].map(supp.groupby(['building_id'])['meter'].unique()).apply(lambda x : np.array_str(np.sort(x)))
#     supp.to_pickle('supp.pkl')
#     del supp
#     gc.collect()
    
def counter_feature(x,cat):
    supp = pd.read_pkl('supp.pkl')
    for feature in cat:
        print(feature)
        x[feature + '_count'] = x[feature].map(supp[feature].value_counts(dropna=False,normalize =True))
        gc.collect()
    del supp
    return(x)

def pred_test_df(test):
    pred_test = pd.DataFrame({'building_id':test.building_id,'meter':test.meter,'timestamp':test.timestamp,
                          'meter_reading':np.zeros(test.shape[0])})
    test.drop(['timestamp'],axis=1,inplace=True)
    return(test,pred_test)

def outlier_smoother_build(df):    
    print('Beginning Smoothing\n')
    df['s_uid'] = df['building_id'].astype(str) +'_'+\
                        df['meter'].astype(str)

    temp_df = df.groupby(['s_uid'])['meter_reading'].apply(lambda x: int(np.percentile(x,99.5)))
    temp_df = temp_df.to_dict()

    df['s_uid'] = df['s_uid'].map(temp_df)
    changed = sum(df['meter_reading']>df['s_uid'])
    build_change = len(df.loc[df['meter_reading']>df['s_uid'],'building_id'].unique())
    print(f'{changed} row changed\n')
    print(f'{build_change} build changed\n')

    df['meter_reading'] = np.where(df['meter_reading']>df['s_uid'], df['s_uid'], df['meter_reading'])
    
    df["month"] = df["timestamp"].dt.month.astype(np.uint8)
    df['s_uid'] = df['building_id'].astype(str) +'_'+\
                    df['month'].astype(str) +'_'+\
                    df['meter'].astype(str)
    
    temp_df = df.groupby(['s_uid'])['meter_reading'].apply(lambda x: int(np.percentile(x,99.5)))
    temp_df = temp_df.to_dict()

    df['s_uid'] = df['s_uid'].map(temp_df)
    
    changed = sum(df['meter_reading']>df['s_uid'])
    build_change = len(df.loc[df['meter_reading']>df['s_uid'],'building_id'].unique())
    print(f'{changed} row changed\n')
    print(f'{build_change} build changed\n')

    df['meter_reading'] = np.where(df['meter_reading']>df['s_uid'], df['s_uid'], df['meter_reading'])
    del changed, temp_df
    return(df.drop(['s_uid','month'],axis=1))

def loader_train(window = 24,component=1,cols =  ['air_temperature','dew_temperature','feel']):
    print('Reader\n')
    train = pd.read_hdf(f'{path_new}data.h5',key='train')    
    train = reduce_mem_usage_sd(train, obj_to_cat=False)

    clean_timestamps(train)
    print('Bad rows dropping...\n')
    train = bad_rows_dropper(train)
#     train = outlier_smoother_build(train)
    gc.collect()

    weather_train = pd.read_hdf(f'{path_new}data.h5',key='weather_train')
    weather_add = pd.read_hdf(f'{path_new}data.h5',key='weather_test')

    weather_train = weather_train.append(weather_add,ignore_index =True)

    weather_train = add_lag_feature(weather_train,cols = cols,window=window)

    building_metadata = pd.read_hdf(f'{path_new}data.h5',key='building_metadata')
        
    weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp'],format='%Y-%m-%d %H:%M:%S')
    weather_train = aligner_holiday(weather_train)

    weather_train = reduce_mem_usage_sd(weather_train, obj_to_cat=False)
    building_metadata = reduce_mem_usage_sd(building_metadata, obj_to_cat=False)

    print('Merging building_data...\n')
    train = train.merge(building_metadata, left_on = "building_id", right_on = "building_id", how = "left")

    print('Merging weather_data...\n')
    train = train.merge(weather_train, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")
    mask = (train[[f'{x}_mean_lag{window}' for x in cols]].isnull()).sum(axis=1)==len(cols)

    train = train.loc[~mask]
    print('Feature engineer\n')
    train = fe_df(train)
    train = to_del(train)
    
    train['meter_reading_mod'] = train['meter_reading'].copy()
    train['meter_reading_mod'][(train['site_id']==0) & (train['meter']==0)] = train['meter_reading_mod'][(train['site_id']==0) & (train['meter']==0)]*0.2931
    
    train['meter_reading_mod'] = np.log1p(train['meter_reading_mod'])
    train['meter_reading'] = np.log1p(train['meter_reading'])
    
    train = reduce_mem_usage_sd(train, obj_to_cat=False)

    target_mod = train.pop('meter_reading_mod')
    target = train.pop('meter_reading')

    print('temp_square\n')
    train = temp_square_feet(train)


    gc.collect()

    category_column = ['building_id','meter','day','hour','weekend','dayofweek','month','dayofyear','country','Holiday','floor_count',
                       'site_id','primary_use','diff_meter','building_meter']
    category_column = [x for x in category_column if x in train.columns]

    print('as int\n')
    for col in tqdm_notebook(category_column):
        train[col] = (train[col].astype('category')).astype(np.int16)
        
    train = reduce_mem_usage_sd(train, obj_to_cat=False)

    mask = (train.site_id==0) & (train.meter==0) & (train.timestamp <='2016-05-20') 
    target = target.loc[~mask]
    train = train.loc[~mask]
    
    train['timestamp'].to_hdf('dataset.h5',key='timestamp',complib='blosc', complevel=9)
    train.drop('timestamp',axis=1,inplace=True)

    train.reset_index(drop=True,inplace=True)
    target.reset_index(drop=True,inplace=True)
    del weather_train,weather_add,building_metadata,mask
    
    col = debugger_train(train,target)

    train.to_hdf('dataset.h5',key='train',complib='blosc', complevel=9)
    target.to_hdf('dataset.h5',key='target',complib='blosc', complevel=9)
    target_mod.to_hdf('dataset.h5',key='target_site0_mod',complib='blosc', complevel=9)

    pd.DataFrame({'column':train.columns}).to_hdf('dataset.h5',key='column',complib='blosc', complevel=9)
    pd.DataFrame({'cat_column':category_column}).to_hdf('dataset.h5',key='category_column',complib='blosc', complevel=9)

    return(col)

def loader_test(col_check,window = 24,component=1,cols =  ['air_temperature','dew_temperature','feel']):
    print('Reader\n')
    test = pd.read_hdf(f'{path_new}data.h5',key='test')
    test = reduce_mem_usage_sd(test, obj_to_cat=False)

    clean_timestamps(test)
    gc.collect()
    
    weather_train = pd.read_hdf(f'{path_new}data.h5',key='weather_train')
    weather_add = pd.read_hdf(f'{path_new}data.h5',key='weather_test')

    weather_test = weather_train.append(weather_add,ignore_index =True)

    weather_test = add_lag_feature(weather_test,cols = cols,window=window)

    building_metadata = pd.read_hdf(f'{path_new}data.h5',key='building_metadata')

    weather_test['timestamp'] = pd.to_datetime(weather_test['timestamp'],format='%Y-%m-%d %H:%M:%S')
    weather_test = aligner_holiday(weather_test)


    test = reduce_mem_usage_sd(test, obj_to_cat=False)
    
    weather_test = reduce_mem_usage_sd(weather_test, obj_to_cat=False)
    building_metadata = reduce_mem_usage_sd(building_metadata, obj_to_cat=False)

    print('Merging building_data...\n')
    test = test.merge(building_metadata, left_on = "building_id", right_on = "building_id", how = "left")

    print('Merging weather_data...\n')
    test = test.merge(weather_test, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")
    print('Feature engineer\n')
    test = fe_df(test)
    test = to_del(test)

    test = temp_square_feet(test)
    test = reduce_mem_usage_sd(test, obj_to_cat=False)

    
    gc.collect()

    category_column = ['building_id','meter','day','hour','weekend','dayofweek','month','dayofyear','country','Holiday','floor_count',
                       'site_id','primary_use','diff_meter','building_meter']
    category_column = [x for x in category_column if x in test.columns]

    for col in tqdm_notebook(category_column):
        test[col] = (test[col].astype('category')).astype(np.int16)

    test = reduce_mem_usage_sd(test, obj_to_cat=False)
    test['timestamp'].to_hdf('dataset.h5',key='timestamp_test',complib='blosc', complevel=9)
    test.drop(['timestamp'],axis=1,inplace=True) 

    debugger_test(test,col_check)
    test.to_hdf('dataset.h5',key='test',complib='blosc', complevel=9)

    del weather_test,building_metadata

In [5]:
%%time
from multiprocessing import Process, Queue

def loader_ram_aware(q):
    q.put(loader_train())

queue = Queue()
p = Process(target=loader_ram_aware, args=(queue,))
p.start()
p.join()
col = queue.get()

Reader




Mem. usage decreased from 2699.14 Mb to 2120.75 Mb (21.4% reduction)
Bad rows dropping...

Load data...

Find rows...

Number of rows dropped: 986260



Mem. usage decreased from 61.83 Mb to 36.14 Mb (41.6% reduction)



Mem. usage decreased from  0.06 Mb to  0.03 Mb (56.8% reduction)
Merging building_data...

Merging weather_data...

Feature engineer




Mem. usage decreased from 2473.90 Mb to 2400.54 Mb (3.0% reduction)
temp_square

as int




Mem. usage decreased from 2473.90 Mb to 2327.19 Mb (5.9% reduction)
CPU times: user 332 ms, sys: 288 ms, total: 620 ms
Wall time: 8min 44s


In [6]:
%%time
loader_test(col_check=col)


Reader




Mem. usage decreased from 5249.10 Mb to 4056.12 Mb (22.7% reduction)



Mem. usage decreased from 1471.34 Mb to 1471.34 Mb (0.0% reduction)



Mem. usage decreased from 61.83 Mb to 36.14 Mb (41.6% reduction)



Mem. usage decreased from  0.06 Mb to  0.03 Mb (56.8% reduction)
Merging building_data...

Merging weather_data...

Feature engineer




Mem. usage decreased from 3658.47 Mb to 3658.47 Mb (0.0% reduction)



Mem. usage decreased from 3976.59 Mb to 3658.47 Mb (8.0% reduction)
CPU times: user 11min 9s, sys: 4min 26s, total: 15min 36s
Wall time: 13min 55s
